<a href="https://colab.research.google.com/github/smthomas1704/restoration-rag/blob/main/functional_trait_rag_with_jina_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download
Download the zipped literature file from this Google Drive location

In [ ]:
!git clone https://github.com/smthomas1704/restoration-rag.git

!pip install -r restoration-rag/requirements.txt
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install gdown==v4.6.3
!pip install openai
!pip install langchain_experimental

!gdown https://drive.google.com/file/d/10_inKhFuY5O8Sel88ZvlTqODsbbh4ula/view?usp=drive_link -O /content/functional_trait_literature_zipped.zip --fuzzy

!unzip /content/functional_trait_literature_zipped.zip

# Chunk and store

In this portion, we'll chunk all the files into smaller paragraphs and use that for generating embeddings. We'll separately store the chunks so we can use it later, without having to re-download all the literature again.

### References/Notes related to chunking
1. https://openai.com/blog/new-and-improved-embedding-model
2. text-embedding-ada-002 is the best model for text embedding generation
3. https://www.pinecone.io/learn/chunking-strategies/
4. https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pypdf

### Possible strategies for chunking
1. LaTex: LaTeX is a document preparation system and markup language often used for academic papers and technical documents. By parsing the LaTeX commands and environments, you can create chunks that respect the logical organization of the content (e.g., sections, subsections, and equations), leading to more accurate and contextually relevant results.
2. Latex taxes a string as input, so we will need to read
3. Most of these academic papers are written in Latex and then converted to PDF. Our best bet would be to convert the PDF to Latex format first and then use the Latex based chunker to chunk things. This way paragraphs and related information will be together and contextualized.
4. On the other hand its not a guarantee that the document was first written in LaTex.

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_PATH = 'functional_trait_literature' #Your root data folder path

# Copied from "Using plant functional traits to guide restoration: A case study in California coastal grassland B. SANDEL,1,4, J. D. CORBIN,2 AND M. KRUPA3"
documents = """
Functional traits and trait-based community ecology theory can provide a basis for predicting the success of a restoration treatment in a particular community (Lavorel and Garnier 2002, Pywell et al. 2003, da Silveira Pontes et al. 2010, Roberts et al. 2010), helping to ensure that treatments are applied only where they will be most beneficial. Some research to date has used functional traits to predict species’ success in restoration treatments (Pywell et al. 2003, Rob- erts et al. 2010), with considerable success. For example, Pywell et al. (2003) have shown that seed germination rates and seedling growth rates are both positively associated with species success in sown restorations. These studies have suggested which species are most likely to successfully establish when creating restored communities and indicate, in a general sense, that the relative competitive and colonizing abilities of species may be predicted from their traits. However, little research has addressed how different restoration schemes, mediated by plant traits, can alter these relative abilities. For example, traits that might be advantageous under high-nutrient conditions, such as high seedling growth rates, may be disadvantageous under restored low-nutrient conditions. Such dependencies are predicted based on trait-envi- ronment relationships and can have potentially important consequences for restoration success, but have received little testing in a restoration context.\n\n
Nitrogen enrichment of terrestrial habitats via atmospheric N deposition (Galloway et al. 1995, Fenn et al. 2003) and invasion of N-fixing plants (Daehler 1998) is a significant component of anthropogenic global change (Vitousek et al. 1997a). Elevated soil N can facilitate the invasion of non-native species and complicate efforts to restore native biodiversity (Maron and Connors 1996, Vitousek et al. 1997b, Perry et al. 2010). In this study, we focus on two strategies designed to restore habitats where species invasion is associ- ated with elevated N inputs (Perry et al. 2010): carbon (C) addition and the mowing and removal of above-ground biomass. Both of these\n
techniques have been applied in a variety of systems to control non-native species and to favor native biodiversity (Maron and Jefferies 2001, Van Dyke et al. 2004, Blumenthal 2009, Alpert 2010, Perry et al. 2010). Carbon addition is intended to stimulate bacterial growth, leading to immobilization of N in microbial biomass and a temporary decrease in plant-available N (Baer et al. 2003, Averett et al. 2004, Corbin and D’Anto- nio 2004). Mowing is meant to remove above- ground biomass, thereby reducing the competi- tive advantage of early-germinating or highly productive species, while removing mowed clippings exports N from the system (Zobel et al. 1996, Wilson and Clark 2001).\n\n
Experimental tests of these methods have produced mixed results. While both treatments have been shown to decrease the exotic-to-native ratio in some ecosystems, in others native species have experienced no benefit or those benefits have been limited to a few species (e.g., Maron and Jefferies 2001, Cione et al. 2002, Van Dyke et al. 2004, Blumenthal 2009). A likely explanation for this is that the simple classification of species into ‘‘native’’ and ‘‘exotic’’ categories may pro- vide relatively little functional information. Di- rect measurement and description of plant strategies may enable better predictions for species responses to these, and other, restoration treatments (Pywell et al. 2003, Eschen et al. 2006).\n\n
One useful system for describing plant strate- gies is based on three major axes quantifying leaf, growth, and reproductive strategies (Westoby 1998). Plant species are arrayed along the leaf axis according to the speed at which they obtain returns on leaf investments (Chapin 1980, Reich et al. 1997, Westoby et al. 2002, Wright et al. 2004). Species with fast returns exhibit relatively rapid turnover of leaf biomass and tend to produce thin, low density leaves with high specific leaf area (SLA) (Wright and Westoby 2000, Westoby et al. 2002, Wright et al. 2004). The growth axis describes the structural characteris- tics of a plant, of which height is a key indicator. Tall species invest in structural biomass that improves their access to light, while short species can allocate a larger portion of their biomass towards photosynthetically active tissue, which is advantageous when light is readily available (Falster and Westoby 2003). The reproductive strategy axis can be related to seed mass, as larger-seeded species typically have a higher probability of germinating and can outcompete smaller-seeded species. However, plants that produce large seeds typically produce smaller numbers of seeds, limiting their ability to colonize available microsites (e.g., Turnbull et al. 1999, Henery and Westoby 2001).\n\n
"""

# For testing only
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=500)
test_splits = text_splitter.split_text(documents)

print(len(test_splits))
print(test_splits[0])
print(test_splits[1])


# Split the files
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=200)
prod_splits = text_splitter.split_documents(documents)

print(len(prod_splits))
print(documents[0])

2
Functional traits and trait-based community ecology theory can provide a basis for predicting the success of a restoration treatment in a particular community (Lavorel and Garnier 2002, Pywell et al. 2003, da Silveira Pontes et al. 2010, Roberts et al. 2010), helping to ensure that treatments are applied only where they will be most beneficial. Some research to date has used functional traits to predict species’ success in restoration treatments (Pywell et al. 2003, Rob- erts et al. 2010), with considerable success. For example, Pywell et al. (2003) have shown that seed germination rates and seedling growth rates are both positively associated with species success in sown restorations. These studies have suggested which species are most likely to successfully establish when creating restored communities and indicate, in a general sense, that the relative competitive and colonizing abilities of species may be predicted from their traits. However, little research has addressed how diff

## Now we will generate embeddings using different models for comparison
First one is jina-embedding-l-en-v1

In [ ]:
!mkdir "/content/vectorstore"
DB_JINA_EMBEDDING_PATH = 'vectorstore/db_jina-embedding-l-en-v1'

embeddings = HuggingFaceEmbeddings(model_name='jinaai/jina-embedding-b-en-v1',
                                       model_kwargs={'device': 'cuda'})

db = FAISS.from_documents(prod_splits, embeddings)
serialized_bytes = db.serialize_to_bytes()
with open("/content/vectorstore/serialized_db.txt", "wb") as binary_file:
    # Write bytes to file
    binary_file.write(serialized_bytes)

db.save_local(DB_JINA_EMBEDDING_PATH)

In [ ]:
docs = await db.asimilarity_search("I am designing a tropical dry forest restoration in an open field with no remaining tree cover. Should I plant species with higher or lower wood density to maximum initial survival?")
print(len(docs))
print(docs[0].page_content)
print(docs[1].page_content)
print(docs[2].page_content)
print(docs[3].page_content)

In [48]:
from google.colab import userdata
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import tiktoken
import os

OPENAI_API_KEY = userdata.get('COLABORATIVE_EARTH_KEY')
llm_model = "gpt-3.5-turbo"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
retriever = db.as_retriever(
    search_kwargs={"k": 6}
)

custom_template = """You are an AI assistant for assisted restoration papers.
You are given the following extracted parts of a long document and a question. Provide a conversational answer with reference to the document.
You should only use names that are explicitly listed as a source in the context. Do NOT make up a hyperlink that is not listed.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""

custom_prompt = PromptTemplate(
    template=custom_template,
    input_variables=["question", "context"],
)


conversation_chain_with_reference_prompt = ConversationalRetrievalChain.from_llm(
        llm=llm,
        # chain_type="stuff" will go through everything.
        # chain_type="refine",
        chain_type="stuff",
        retriever=retriever,
        verbose=True,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": custom_prompt}
)


conversation_chain_without_reference = ConversationalRetrievalChain.from_llm(
        llm=llm,
        # chain_type="stuff" will go through everything.
        chain_type="refine",
        # chain_type="stuff",
        retriever=retriever,
        # return_source_documents=True,
        verbose=True,
        memory=memory,
        # combine_docs_chain_kwargs={"question_prompt": custom_prompt, "refine_prompt": custom_prompt}
)

###TODO
1. Add a cross encoder after the retrieval stage to re-rank the results before feeding it to the API.
2. Alternately, or maybe along with this, we may also want to combine several chunks to send as context, depending on the context lenght.
3. Or perhaps we also chunk in small portions and combine results to send to OpenAI. This way answers can be formed based on chunks from different papers and different sections.
4. Update this RAG to cite sources from the context provided. Refence: https://blog.langchain.dev/langchain-chat/

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [50]:
query = "Which functional traits are useful to measure to understand species performance in restoration plantings?"
# result = conversation_chain_with_reference_prompt({"question": query})
result = conversation_chain_without_reference({"question": query})
answer = result["answer"]

answer



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Which functional traits are useful to measure to understand species performance in restoration plantings?
Assistant: The document mentions that functional traits, which are morphological, physiological, or phenological characteristics linked to plant survival, growth, or reproduction, are useful to measure in order to understand species performance in restoration plantings (Violle et al., 2007). By examining functional traits, we can understand the mechanisms behind a species' performance and extrapolate the results to other native species with similar traits (Martínez-Garza, Bongers, & Poorter, 2013). This understanding aids in the design of restoration plantings that achieve specific outcomes, such as rapid carbon sequestration or restoration of soil fertilit

'In the context of understanding species performance in restoration plantings, various functional traits can be measured. These traits are important in promoting survival within degraded landscapes and predicting tropical plant growth under different environmental conditions. Some of the functional traits that can be considered include morphological traits (e.g., plant size, leaf area), physiological traits (e.g., water-use efficiency), and biochemical traits (e.g., nutrient uptake efficiency). Additionally, integrating local knowledge of species life-history traits, such as successional status, can improve restoration outcomes and ensure the inclusion of culturally and economically important species in the design process.\n\nIn tropical dry forests, which are highly threatened and a priority for restoration efforts, seedlings are exposed to strong dry seasons in the initial years of development. Therefore, traits related to drought tolerance, such as water-use efficiency, may be parti